O objetivo do código aqui é imputar valores das séries de consumo energético, que começam em jan/2004,
e preciso de valores indo até set/2003

In [1]:
import pandas as pd
import utils
from sktime.transformations.series.impute import Imputer

In [6]:
df = pd.read_csv("dados/dados_crus2003.csv", sep=";", decimal=",")
col_to_keep = ["month", "energia_total", "energia_residencial", "energia_industrial", "energia_comercial", "energia_outros"]
df = df.drop(columns=df.columns.difference(col_to_keep))
df = df.drop([i for i in range(8)]) # Ignora valores até setembro
df = utils.index_to_datetime(df)
df = df.reset_index()
df = df.drop(columns=['index'])
df.head(5)

,month,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros
0,2003-09-01,NaN,NaN,NaN,NaN,NaN
1,2003-10-01,NaN,NaN,NaN,NaN,NaN
2,2003-11-01,NaN,NaN,NaN,NaN,NaN
3,2003-12-01,NaN,NaN,NaN,NaN,NaN
4,2004-01-01,26978.358,6780.68,12008.852,4244.837,3943.989


In [7]:
transformer = Imputer() # Usa regressão linear
df = transformer.fit_transform(df)
df.head(5)

C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\transformations\series\impute.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][na_index] = self._forecaster.predict(fh=fh, X=y)
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\transformations\series\impute.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][na_index] = self._forecaster.predict(fh=fh, X=y)
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFo

,month,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros
0,2003-09-01,28512.930262,6437.899779,13685.568722,4405.625965,3983.835892
1,2003-10-01,28578.224388,6467.379992,13691.290782,4422.109034,3997.444676
2,2003-11-01,28643.518515,6496.860205,13697.012842,4438.592103,4011.053460
3,2003-12-01,28708.812641,6526.340418,13702.734901,4455.075172,4024.662244
4,2004-01-01,26978.358000,6780.680000,12008.852000,4244.837000,3943.989000


In [8]:
f = lambda df, i, col_name: df.loc[i, col_name] - df.loc[i - 1, col_name]
transf_df = utils.transform_dataframe(df, f)
transf_df.head(5)

,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros
month,,,,,
2003-10-01,65.294126,29.480213,5.722060,16.483069,13.608784
2003-11-01,65.294126,29.480213,5.722060,16.483069,13.608784
2003-12-01,65.294126,29.480213,5.722060,16.483069,13.608784
2004-01-01,-1730.454641,254.339582,-1693.882901,-210.238172,-80.673244
2004-02-01,-470.187000,-351.442000,132.565000,-97.562000,-153.748000


In [9]:
results = utils.test_for_stationarity_kpss(transf_df)

C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  res = kpss(
C:\Users\guisf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\param_est\stationarity\_statsmodels.py:253: InterpolationWarning: The test statistic is outside of the range of 

In [11]:
utils.print_stationary_series(results)

energia_total
energia_residencial
energia_industrial
energia_comercial
energia_outros


In [ ]:
transf_df.to_csv("dados/energia_estacionaria.csv", sep=";", decimal=",")